In [ ]:
import ollama

import faiss
import pickle
import os
from sentence_transformers import SentenceTransformer
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch


RAG Pipeline Testing

In [36]:
ollama.pull('hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [21]:
def load_retriever(
    index_path: str,
    chunks_path: str,
    model_path: str = 'C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\embedding_model'
):
    # Load sentence transformer model
    if os.path.exists(model_path):
        model = SentenceTransformer(model_path)
    else:
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        model.save(model_path)

    # Load FAISS index
    index = faiss.read_index(index_path)

    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)

    return model, index, chunks_list


In [23]:
model, index, chunks_list = load_retriever(
        index_path="C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\vector_db\\paragraphs\\urdu_faiss_index_for_100_para.index",
        chunks_path="C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\data_storage\\parachunks\\urdu_chunks_for_100_para.pkl"
    )


In [24]:
def retrieve_documents(query, model, index, chunks_list, k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)
    retrieved_chunks = [chunks_list[i] for i in I[0]]
    return retrieved_chunks

In [14]:
model_path = "C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\Lughat_model"

gen_tokenizer = AutoTokenizer.from_pretrained(model_path)
gen_model = AutoModelForCausalLM.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  2.18it/s]


In [34]:
def generate_using_lughat(context, query, max_new_tokens=200):
    prompt = f"""آپ کو ایک سوال اور اس سے متعلق ایک سیاق و سباق دیا گیا ہے۔ براہ کرم سیاق و سباق کا بغور مطالعہ کریں اور اسی کی بنیاد پر درست، مختصر اور جامع جواب دیں۔

### سوال:
{query}

### سیاق و سباق:
{context}

### جواب:
"""
    inputs = gen_tokenizer(prompt, return_tensors="pt").to(gen_model.device)
    outputs = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.split("### جواب:")[-1].strip()



In [31]:
import ollama

def generate_using_alif(query, context, alif_model='hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16'):


    prompt = f"""آپ کو ایک سوال اور اس سے متعلق ایک سیاق و سباق دیا گیا ہے۔ براہ کرم سیاق و سباق کا بغور مطالعہ کریں اور اسی کی بنیاد پر درست، مختصر اور جامع جواب دیں۔

### سوال:
{query}

### سیاق و سباق:
{context}

### جواب:
"""

    response = ollama.chat(
        model=alif_model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    return response['message']['content']


In [ ]:
def rag_pipeline(query: str, k=3) -> str:

    retrieved_chunks = retrieve_documents(query, model, index, chunks_list, k=k)
    # answer = generate_using_lughat(query, retrieved_chunks)
    answer=generate_using_alif(query, retrieved_chunks)
    return answer


In [33]:
question = "سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بناء پر سیری بی کونٹٹ) لیگ کا 86 واں سیزن ہے ، جس میں مجموعی طور پر 22 ٹیمیں مقابلہ کر رہی ہیں: 15 2016 - 17 سیزن سے واپس آرہی ہیں ، 2016 - 17 سیری بی ، جو اسپانسرشپ کی وجوہات کی بناء پر سیری بی کونٹٹٹ کے نام سے جانا جاتا ہے ، اور اس کے قیام کے بعد سے 85 واں سیزن تھا۔"
ans=rag_pipeline(question)
ans

retrieve_documents: ['سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی یورو بی) کا مقابلہ کیا گیا تھا 82 ویں سیزن اس کے قیام کے بعد سے ، مجموعی طور پر 22 ٹیموں نے لیگ میں مقابلہ کیا: جن میں سے 15 2012-2013 کے سیزن سے واپس آرہی تھیں ، جن میں سے 4 کو لیگا پرو پریما ڈویژن سے', 'سیری بی (اسپانسرشپ کی وجوہات کی بناء پر سیری بی کے نام سے جانا جاتا ہے) اس کی بنیاد پر 81 ویں سیزن ہے ، جس میں مجموعی طور پر 22 ٹیمیں مقابلہ کریں گی: جن میں سے 15 2011 سے واپس آ رہی ہیں ، جن میں سے 4 لیگ پرو پریما ڈویژن سے ترقی یافتہ ہیں ، اور سیری', 'سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی) کا مقابلہ کرنے والی لیگ کی 86 ویں سیزن ہے۔ اس میں مجموعی طور پر 22 ٹیمیں حصہ لے رہی ہیں: 15 2016 سے واپس آ رہی ہیں ، 4 لیگا پرو سے ترقی یافتہ ہیں ، اور 3 سیری اے سے نیچے گر چکے ہیں۔']


'سری بی (اسپانسرشپ کی وجوہات کی بنا پر سری بی کونٹیننٹ) لیگ کا 86 واں سیزن 2017 - 17 (اسپانسر شپ کی وجوہات کی بناء پر سیری بی کونٹیننٹ کے نام سے جانا جاتا ہے)، جو اگست 2017 میں شروع ہوا تھا، اور اس وقت تک چل رہا ہے.'

RAG Pipeline testing on Dataset


In [41]:
import pandas as pd



# Load only the required columns
df = pd.read_csv('C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\Dataset\\Hotpotqa\\1000_paras_100_queries\\translated_dataset_100_qna.csv', usecols=[
    'level', 'translated_question', 'translated_answer', 'translated_retrieved_sentences'
])

# Rename the column
df.rename(columns={'translated_retrieved_sentences': 'translated_context'}, inplace=True)

# Optional: View the result
print(df.head())

  level                                translated_question translated_answer  \
0  easy  سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...         1929ء میں   
1  easy  "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...       کرپٹوولوجسٹ   
2  easy  "جراسک پارک کے اداکار ڈیوڈ ہنری ہوانگ نے ""دی ...        بی ڈی وانگ   
3  easy  کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...        دادا سمپسن   
4  easy     کون تھا ایک حصہ S#arp، لی Ji-hye یا کرٹس رائٹ؟          لی جی ہے   

                                  translated_context  
0  لیگ میں مجموعی طور پر 22 ٹیمیں حصہ لے رہی ہیں:...  
1  ایلکلر ڈکن (انگریزی: Alec Dakin) (۳ اپریل ۱۹۱۲...  
2  "جیرز زکس نے اس فلم کی ہدایت کاری کی تھی جس می...  
3  "سیمپسنز کے 22 ویں سیزن کی دوسری قسط "" لون اے...  
4  لی جی ہیو (پیدائش 11 جنوری ، 1980) ایک جنوبی ک...  


In [ ]:
import time

# Add empty columns first (optional, for visibility)
df['retrieved_context'] = ""
df['final_answer'] = ""
df['retriever_time'] = 0.0
df['generator_time'] = 0.0
df['total_time'] = 0.0

# Start processing rows
for i, row in df.iterrows():
    query = row['translated_question']
    
    # Track total processing time
    total_start = time.time()
    
    # ⏱️ Retrieve
    retriever_start = time.time()
    retrieved_chunks = retrieve_documents(query, model, index, chunks_list, k=3)
    retriever_end = time.time()
    
    retrieved_context = "\n".join(retrieved_chunks)  # Combine if it's a list
    df.at[i, 'retrieved_context'] = retrieved_context
    df.at[i, 'retriever_time'] = retriever_end - retriever_start

    # ⏱️ Generate
    generator_start = time.time()
    final_answer = generate_using_alif(query, retrieved_context)
    generator_end = time.time()
     
    df.at[i, 'final_answer'] = final_answer
    df.at[i, 'generator_time'] = generator_end - generator_start

    # ⏱️ Total Time
    df.at[i, 'total_time'] = time.time() - total_start

    # Print progress (how many processed so far)
    print(f"Processed {i + 1}/{len(df)} rows")

# Optionally save the results to a new CSV
df.to_csv('C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\results\\Simple RAG QNA results\\simple_rag_qna_results.csv', index=False,encoding="utf-8-sig")


Processed 1/98 rows
Processed 2/98 rows
Processed 3/98 rows
Processed 4/98 rows
Processed 5/98 rows
Processed 6/98 rows
Processed 7/98 rows
Processed 8/98 rows
Processed 9/98 rows
Processed 10/98 rows
Processed 11/98 rows
Processed 12/98 rows
Processed 13/98 rows
Processed 14/98 rows
Processed 15/98 rows
Processed 16/98 rows
Processed 17/98 rows
Processed 18/98 rows
Processed 19/98 rows
Processed 20/98 rows
Processed 21/98 rows
Processed 22/98 rows
Processed 23/98 rows
Processed 24/98 rows
Processed 25/98 rows
Processed 26/98 rows
Processed 27/98 rows
Processed 28/98 rows
Processed 29/98 rows
Processed 30/98 rows
Processed 31/98 rows
Processed 32/98 rows
Processed 33/98 rows
Processed 34/98 rows
Processed 35/98 rows
Processed 36/98 rows
Processed 37/98 rows
Processed 38/98 rows
Processed 39/98 rows
Processed 40/98 rows
Processed 41/98 rows
Processed 42/98 rows
Processed 43/98 rows
Processed 44/98 rows
Processed 45/98 rows
Processed 46/98 rows
Processed 47/98 rows
Processed 48/98 rows
P